<a href="https://colab.research.google.com/github/jkranyak/birdscompetitionkaggle/blob/kaggle/Kaggle_pre_loaded_model_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'birdclef-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F70203%2F8068726%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240411%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240411T234502Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D46e0e22533c4fcd389c01d0e4a59680f0195c48981d89b38fbcc56169788c4e2ec0cd598173d4b060798187fc098d9ae8be44f928e19332fd7a04dd9291b7cd5812e6ccb7a55cde8fb478dd218fbb7838d1668800383c690cadf71fdb848aef8fc877fe66aa901c88f53b367b5159910d89ce41d7d6e5efaa398ab4edb3a3698eb791cf66685b93ce0b06ccbe72381b605d5a1cd8666e4a5b0c3deed470d2e40bcebf7459e89a08001c6c206049a51e26d8448f234521076fa17a96a87092b56e9f57708917dce91638fcc740d304c9522406f9230270c574117522f32a1760132f49b58bdb9ccae232fd02fe940370249bd14d548dc193e0be81f80f0badebf'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==============                                    ] 6795264000 bytes downloaded

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load the pre-trained model from TensorFlow Hub
import kagglehub

# Download latest version
path = kagglehub.model_download("google/bird-vocalization-classifier/tensorFlow2/bird-vocalization-classifier")

print("Path to model files:", path)


In [ ]:
# Assuming the file path is correct and the file exists
metadata_path = '/kaggle/input/birdclef-2024/train_metadata.csv'
metadata = pd.read_csv(metadata_path)

# Display the first few rows of the dataframe
display(metadata.head())

# Display basic information about the dataframe
print(metadata.info())

In [ ]:
import pandas as pd

# Load the metadata
metadata_path = '/kaggle/input/birdclef-2024/train_metadata.csv'
metadata = pd.read_csv(metadata_path)

# Apply the expanded geographic filter for the Western Ghats and surrounding migratory areas
western_ghats_birds = metadata[
    (metadata['latitude'].between(8, 20)) &
    (metadata['longitude'].between(72, 80))
]

# Handle missing values if necessary (drop or impute)
western_ghats_birds = western_ghats_birds.dropna(subset=['latitude', 'longitude'])

# Print the new DataFrame to verify the filter and see the first few rows
print(western_ghats_birds.head())
print(western_ghats_birds.info())

# Optional: Save this DataFrame to a new CSV for easier access or further analysis
output_path = '/kaggle/working/western_ghats_birds.csv'
western_ghats_birds.to_csv(output_path, index=False)
print(f"DataFrame saved to {output_path}")


In [ ]:
import pandas as pd

# Assuming df is already loaded
df_path = '/kaggle/working/western_ghats_birds.csv'
df = pd.read_csv(df_path)

# Drop unnecessary columns
df = df.drop(['author', 'license', 'rating', 'url', 'secondary_labels', 'type'], axis=1)

# Optionally, save the trimmed DataFrame for further use
trimmed_df_path = '/kaggle/working/western_ghats_birds_trimmed.csv'
df.to_csv(trimmed_df_path, index=False)

print("Trimmed DataFrame saved:", trimmed_df_path)
print(df.head())  # Show the first few entries of the trimmed DataFrame


In [ ]:
import librosa

base_audio_dir = '/kaggle/input/birdclef-2024/train_audio'

def load_audio_data(row):
    file_path = os.path.join(base_audio_dir, row['filename'])
    audio, sr = librosa.load(file_path)
    return audio, sr

# Example of loading audio for the first few filtered entries
for index, row in filtered_metadata.head().iterrows():
    audio, sr = load_audio_data(row)
    print(f"Loaded audio from {row['filename']} with sample rate {sr}")


In [ ]:
import os
import shutil
import pandas as pd

# Load the DataFrame
df_path = '/kaggle/working/western_ghats_birds.csv'
western_ghats_birds = pd.read_csv(df_path)

# Base directory where original audio files are stored
base_audio_dir = '/kaggle/input/birdclef-2024/train_audio'

# Directory to store the copied audio files
target_dir = '/kaggle/working/processed_embeddings/western_ghats_bird_calls'
os.makedirs(target_dir, exist_ok=True)

# Initialize a counter for copied files
files_copied = 0

# Iterate through the DataFrame and copy files
for idx, row in western_ghats_birds.iterrows():
    src_path = os.path.join(base_audio_dir, row['filename'])
    if os.path.exists(src_path):
        # Ensure the target subdirectory exists
        subfolder_path = os.path.join(target_dir, os.path.dirname(row['filename']))
        os.makedirs(subfolder_path, exist_ok=True)

        # Define the destination path
        dest_path = os.path.join(target_dir, row['filename'])

        # Copy the file, replacing it if it already exists
        shutil.copy2(src_path, dest_path)  # Use copy2 for better handling of metadata and overwriting
        files_copied += 1  # Increment the counter
        print(f"Copied and replaced if existing: {src_path} to {dest_path}")
    else:
        print(f"File not found: {src_path}")

# Print the total number of files copied
print(f"All specified audio files have been processed. Total files copied or replaced: {files_copied}")


In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load the pre-trained model from TensorFlow Hub
model_url = '/root/.cache/kagglehub/models/google/bird-vocalization-classifier/tensorFlow2/bird-vocalization-classifier/8'
model = hub.load(model_url)

def load_and_preprocess_audio(file_path, sr=32000, duration=5):
    """ Load and preprocess audio to have consistent duration and sample rate """
    audio, _ = librosa.load(file_path, sr=sr, duration=duration)
    if len(audio) < sr * duration:
        audio = np.pad(audio, (0, sr * duration - len(audio)), 'constant')
    return audio

def extract_features(audio):
    """ Extract embedding features from the pre-trained model """
    audio_tensor = tf.convert_to_tensor([audio], dtype=tf.float32)
    outputs = model.signatures['serving_default'](audio_tensor)
    embeddings = outputs['embedding'].numpy()
    return embeddings.flatten()

# Load metadata and filtered audio file paths
df_path = '/kaggle/working/western_ghats_birds.csv'
df = pd.read_csv(df_path)

# Directory with the relevant audio files
audio_dir = '/kaggle/working/processed_embeddings/western_ghats_bird_calls'

features = []
labels = []

# Process each file in the DataFrame
for _, row in df.iterrows():
    audio_path = os.path.join(audio_dir, row['filename'])
    if os.path.exists(audio_path):
        audio_data = load_and_preprocess_audio(audio_path)
        features.append(extract_features(audio_data))
        labels.append(row['primary_label'])

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# Train a RandomForest classifier
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_train, y_train)

# Evaluate the classifier
predictions = classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print("Model trained and evaluated.")
print(f"Accuracy: {accuracy}")



In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Directory where the segmented audio files will be stored
segmented_audio_dir = '/kaggle/working/western_ghats_audio_segments'
os.makedirs(segmented_audio_dir, exist_ok=True)  # Ensure the directory exists

# Function to load audio, segment it into 5-second clips, and save the segments
def load_and_segment_audio(file_path, output_dir, filename, sr=32000, segment_length=5):
    audio, _ = librosa.load(file_path, sr=sr)
    segments = []
    segment_count = 0
    for start in range(0, len(audio), sr * segment_length):
        end = start + sr * segment_length
        if end <= len(audio):
            segment_path = os.path.join(output_dir, f"{filename}_segment_{segment_count}.wav")
            librosa.output.write_wav(segment_path, audio[start:end], sr)
            segments.append(segment_path)
            segment_count += 1
    return segments

# Function to extract features using the pre-trained model
def extract_features(model, audio_path):
    audio_data, _ = librosa.load(audio_path, sr=32000)
    audio_tensor = tf.constant(audio_data, dtype=tf.float32)
    audio_tensor = tf.reshape(audio_tensor, [1, -1])  # Reshape for batch dimension
    outputs = model.signatures['serving_default'](inputs=audio_tensor)
    features = outputs['embedding']
    return features.numpy()

# Load the pre-trained model
model_url = '/root/.cache/kagglehub/models/google/bird-vocalization-classifier/tensorFlow2/bird-vocalization-classifier/8'
feature_extractor = hub.load(model_url)

# Load the metadata for Western Ghats bird calls
df_path = '/kaggle/working/western_ghats_birds.csv'
df = pd.read_csv(df_path)

# Prepare data arrays
features = []
labels = []

# Process each file in the DataFrame
for index, row in df.iterrows():
    file_path = os.path.join(base_audio_dir, row['filename'])
    if os.path.exists(file_path):
        audio_segments = load_and_segment_audio(file_path, segmented_audio_dir, os.path.splitext(row['filename'])[0])
        for segment_path in audio_segments:
            extracted_features = extract_features(feature_extractor, segment_path)
            features.append(extracted_features.flatten())  # Flatten the features
            labels.append(row['primary_label'])

# Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# Train a RandomForest classifier
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_train, y_train)

# Evaluate the classifier
predictions = classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
